# Pop2Piano - Smart Inference (Google Colab) 🎹🎵

Generate piano covers with **automatic Arabic music detection** and **smart composer selection**.

## Features:
- **Auto-detect** if a song is Arabic and which Maqam it uses
- **Auto-select** the best Arr. Style (composer) from 21 options
- **Maqam post-processing** for Arabic scales
- Works with both Arabic and Western songs

In [ ]:
# @title 1. Install Dependencies 📦
import os
import sys
import subprocess

print("Installing dependencies... ⏳")

# Install Python packages
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "transformers", "essentia", "librosa", "resampy", "pretty_midi", "midi2audio", "scipy", "yt-dlp", "imageio-ffmpeg", "omegaconf", "joblib", "tqdm"])

# Install System packages
os.system("sudo apt-get update -q")
os.system("sudo apt-get install -y -q fluidsynth fluid-soundfont-gm ffmpeg")

print("Done! ✅")

In [ ]:
# @title 2. Clone Repository 🐙
import os

repo_url = "https://github.com/kareemkamal10/pop2piano.git"
repo_name = "pop2piano"

%cd /content

if os.path.exists(repo_name):
    print("Repository already cloned. Pulling latest changes...")
    %cd {repo_name}
    !git pull
else:
    print(f"Cloning {repo_url}...")
    !git clone {repo_url}
    %cd {repo_name}

print(f"Current working directory: {os.getcwd()}")

In [ ]:
# @title 3. Load Model & Upload Audio 🎹
from transformers import Pop2PianoForConditionalGeneration, Pop2PianoProcessor
from google.colab import files
import librosa
import torch

# Load Model
print("Loading Model...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Pop2PianoForConditionalGeneration.from_pretrained("sweetcocoa/pop2piano").to(device)
processor = Pop2PianoProcessor.from_pretrained("sweetcocoa/pop2piano")
print(f"Model Loaded on {device}!")

# Upload Audio File
print("\nPlease upload your audio file (MP3/WAV)...")
uploaded = files.upload()
audio_filename = next(iter(uploaded))
print(f"Processing: {audio_filename}")
audio, sr = librosa.load(audio_filename, sr=44100)
print(f"Duration: {len(audio)/sr:.1f}s")

In [ ]:
# @title 4. Analyze & Smart Generate 🎹✨
from smart_inference import SmartPop2Piano, print_analysis_report, analyze_audio

# Analyze audio first
analysis = analyze_audio(audio, sr)
print_analysis_report(analysis)

# Smart Generate
smart = SmartPop2Piano(model, processor, device=device)

# ╔══════════════════════════════════════════════════════════════╗
# ║  SETTINGS                                                    ║
# ║  maqam: None=auto, or "hijaz", "bayyati", "rast", etc.     ║
# ║  composer: None=auto, or "composer1" through "composer21"   ║
# ╚══════════════════════════════════════════════════════════════╝

result = smart.generate(
    audio=audio,
    sr=sr,
    auto_detect_maqam=True,
    maqam=None,                   # Force maqam here if auto is wrong
    composer=None,                # Force composer here if needed
    auto_select_composer=True,
    apply_post_processing=True,
)

# Save & Download
output_midi_name = "output_midi.mid"
result.best_midi.write(output_midi_name)

print(f"\nBest: {result.best_composer} (score: {result.best_score:.2f})")
if result.applied_maqam:
    print(f"Maqam: {result.applied_maqam}")
print(f"Saved to {output_midi_name}")

files.download(output_midi_name)